# Описание гипотезы
### Проверим предположения
+ есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
+ различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

#### Выполнять задание будем на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

***
#### Ход проверки
Импортируем нужные библиотеки

In [1]:
import pandas as pd

Загружаем данные

In [2]:
purchase_df = pd.read_csv('data/purchase.csv', sep=',')
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')

Выбираем 2018 год

In [3]:
# Создаем фильтр по пользователям, зарегистрировавшихся в 2018 году
mask = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')

regist_2018 = events_df[mask]['user_id'].to_list() # список пользователей зарегистрировавшихся в 2018 году 

events_2018 = events_df[events_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице events
purchase_2018 = purchase_df[purchase_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице purchase


Произведем преобразование типов

In [4]:
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])

Перед объединением датафреймов, переименуем колонки

In [5]:
events_2018 = events_2018.rename(columns={'id':'event_id'})
purchase_2018 = purchase_2018.rename(columns={'id':'purchase_id'})

Сделаем объединенный датафрейм из событий и оплат

In [6]:
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)

Сделаем группы пользователей по сложности тренировок

In [7]:
# Какие уровни сложности бывают?
total_events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Сформируем группы

In [8]:
mask_easy = total_events_df['selected_level'] == 'easy'
mask_medium = total_events_df['selected_level'] == 'medium'
mask_hard = total_events_df['selected_level'] == 'hard'

users_easy_level = total_events_df[mask_easy]['user_id'].unique()
users_medium_level = total_events_df[mask_medium]['user_id'].unique()
users_hard_level = total_events_df[mask_hard]['user_id'].unique()

Посчитаем показатели для каждой группы

In [9]:
# создаем список групп
user_groups = [
    {'easy': users_easy_level},
    {'medium': users_medium_level},
    {'hard': users_hard_level}
]

In [25]:
# пройдемся по списку
for group in user_groups:
    level = list(group.keys())[0] # группа
    print('Сводка по уровню:', level)
    group_users = group[level] # датафрейм группы
    count_of_users_in_group = len(group_users) # количество пользователей в группе
    purchase_df_slice = purchase_2018[purchase_2018['user_id'].isin(group_users)] # оплаты из данной группы
    percent_of_purchase = purchase_df_slice['user_id'].nunique()/count_of_users_in_group # высчитываем % оплативших по группе
    print ('Процент оплативших пользователей, выбравших уровень сложности {}: {:.2%}'.format(level,percent_of_purchase))
    
    # создаем датафрейм который содержит выбор уровня сложности по пользователям данной группы
    level_choice_df = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(group_users))]
    if (level_choice_df['user_id'].value_counts().mean()) == 1: # Пользователь выбрал только один уровень сложности
        level_choice_df = level_choice_df[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'}) # Время выбора уровня сложности
        print('пользователей уровня {} - {}'.format(level, len(level_choice_df)))
        purchase_df_slice_2 = purchase_df_slice[['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'}) # Время когда пользователь оплатил
        merged_df = purchase_df_slice_2.merge(level_choice_df,on='user_id',how='inner') # Соединяем датафреймы
        print(merged_df[merged_df['user_id'] == 27884]['level_choice_time', 'purchase_time'])
        
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['level_choice_time'] # И считаем время между выбором уровня и оплатой
        mean_time = merged_df['timedelta'].mean() # Среднее время между выбором уровня сложности и оплатой
        print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))
        print ('Характеристики времени:')
        print (merged_df['timedelta'].describe())
        print()
    else:
        print ('Выбрано более 1 уровня сложности')

Сводка по уровню: easy
Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%
пользователей уровня easy - 2448


KeyError: ('level_choice_time', 'purchase_time')

## Выводы
Процент оплативших пользователей:
1. Выбравших уровень `easy:` **7,72%**.
2. Выбравших уровень `medium:` **20,86%**
3. Выбравших уровень `hard:` **35,39%**

Значит существует прямая зависимость между пользователями выбравших разный уровень сложности и процентом ими оплат.<br>
`Чем более сложный уровень выбирают пользователи, тем больше процент оплат!`

Среднее время между выбором уровня сложности и моментом первой оплаты:
1. Уровень `easy:` 3 дня и 15 часов
2. Уровень `medium:` 3 дня и 23 часа
3. Уровень `hard:` 3 дня и 7 часов

Дольше всего принимают решение об оплате, пользователи выбравшие уровень `medium`, затем пользователи `easy` и быстрее всего принимают решение пользователи с уровнем `hard`<br>
Временной промежуток у пользователей выбравших разный уровень сложности различается, но разброс не большой.

In [12]:
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2767,NaN,NaN,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
2768,NaN,NaN,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
2769,NaN,NaN,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
2774,NaN,NaN,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0
